In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn import model_selection

In [56]:
from sklearn.model_selection import train_test_split

In [60]:
from sklearn.ensemble import RandomForestRegressor

In [129]:
from sklearn.linear_model import LogisticRegression

In [63]:
from sklearn.metrics import mean_absolute_error

In [6]:
df_target=pd.read_csv('train_centroid_distances.csv',index_col='identifiers')

In [35]:
target=df_target['mean']

In [146]:
df_descriptors=pd.read_csv('train_descriptors.csv',index_col='identifiers')

In [147]:
df_descriptors.head()

,Unnamed: 0,name,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,InchiKey,SMILES
identifiers,,,,,,,,,,,,,,,,,,,,,
COXSUE,11365,Optimised_Molecules_V3.3460,8.773011,8.632564,0,0,14.599164,2.244739,4.388162,14.599164,...,185.062283,8.046186,219,11,54.0,58.0,4.833333,2.777778,UYSCRWBEPUAZRA-UHFFFAOYSA-N,CC(=O)NN=C(C)C1=NCCS1
TUWFAU,58476,Optimised_Molecules_V3.19496,10.813451,10.346393,0,0,18.488707,2.357869,4.715737,18.488707,...,199.110947,7.111105,364,22,70.0,80.0,6.805556,3.583333,RYPSYCXCJSHDJX-UHFFFAOYSA-N,CC(C)N(C)c1ccc(C#N)cc1C#N
DUTHEG,31879,Optimised_Molecules_V3.5074,10.922841,9.584172,0,0,16.685332,2.436274,4.872549,16.685332,...,191.080710,8.307857,271,22,74.0,87.0,5.666667,3.000000,PVRUUABKIQNQFL-UHFFFAOYSA-N,Cc1cc(=O)n2c(N)nc(C)nc2n1
AXUDED08,2885,Optimised_Molecules_V3.988,7.140017,6.873356,0,0,12.857279,2.214320,4.428639,12.857279,...,136.052429,7.558468,121,11,44.0,48.0,3.722222,2.444444,QPJVMBTYPHYUOC-UHFFFAOYSA-N,COC(=O)c1ccccc1
ARAGUV,2660,Optimised_Molecules_V3.795,7.399237,7.308033,0,0,11.638727,2.249778,4.311276,11.638727,...,139.074562,7.319714,125,8,46.0,49.0,4.333333,2.250000,LUFRABHJXNJTNZ-UHFFFAOYSA-N,CC(=O)Nc1cc(C)[nH]n1


In [148]:
df_descriptors.dropna(inplace=True,axis=1)

In [255]:
n_col=df_descriptors.iloc[:,18:34].columns

In [256]:
n_col

Index(['nAromBond', 'nAtom', 'nHeavyAtom', 'nSpiro', 'nBridgehead', 'nHetero',
       'nH', 'nB', 'nC', 'nN', 'nO', 'nS', 'nP', 'nF', 'nCl', 'nBr'],
      dtype='object')

In [257]:
var=df_descriptors.var()

In [258]:
pred_column=var.sort_values(ascending=False)[1:50].index

In [259]:
pred_column=pred_column.append(n_col)

In [260]:
pred_column

Index(['ATS0m', 'ATS4m', 'ATS3m', 'ATSC0m', 'ATS4i', 'ATS5i', 'ATS6i', 'ATS4v',
       'ATS3v', 'ATS5v', 'ATS2m', 'ATS3i', 'ATS2v', 'ATS7i', 'ATS6v', 'ATS5m',
       'ATS8i', 'ATS6m', 'ATS2i', 'ATS7v', 'ATS1v', 'ATSC5m', 'ATSC4m',
       'ATS0v', 'ATS8v', 'ATSC3m', 'ATSC6m', 'ATS1m', 'ATS0Z', 'ATS7m',
       'ATSC2m', 'ATS0i', 'ATS1i', 'ATSC7m', 'ATS4Z', 'ATS3Z', 'ATSC8m',
       'ATSC0Z', 'ATS8m', 'ATSC5v', 'ATSC6v', 'ATS2Z', 'ATSC4v', 'ATS5Z',
       'AATS0m', 'ATSC7v', 'ATSC0v', 'ATS6Z', 'ATSC3v', 'nAromBond', 'nAtom',
       'nHeavyAtom', 'nSpiro', 'nBridgehead', 'nHetero', 'nH', 'nB', 'nC',
       'nN', 'nO', 'nS', 'nP', 'nF', 'nCl', 'nBr'],
      dtype='object')

In [261]:
df=df_descriptors[pred_column]
df.head()

,ATS0m,ATS4m,ATS3m,ATSC0m,ATS4i,ATS5i,ATS6i,ATS4v,ATS3v,ATS5v,...,nH,nB,nC,nN,nO,nS,nP,nF,nCl,nBr
identifiers,,,,,,,,,,,,,,,,,,,,,
COXSUE,2893.425299,2194.932881,2715.297217,1401.437907,5151.343678,5301.706545,6122.390295,5659.668288,6364.304724,5418.328318,...,11,0,7,3,1,1,0,0,0,0
TUWFAU,2332.966431,3396.158023,3726.037218,914.989572,11123.037526,11555.150369,8815.213223,11979.313694,12065.117806,10080.901461,...,13,0,12,3,0,0,0,0,0,0
DUTHEG,2400.205790,3701.031039,3905.512878,810.851632,6885.621507,7001.148205,4510.428125,8177.121069,9467.877591,5844.605240,...,9,0,8,5,1,0,0,0,0,0
AXUDED08,1674.177482,1439.472399,2012.075247,644.354010,4560.942404,3048.950709,2831.649203,4327.265269,5900.133144,2875.344366,...,8,0,8,0,2,0,0,0,0,0
ARAGUV,1719.285450,1388.133143,1640.528952,700.077610,5343.113326,3243.200465,3239.609099,4036.897376,5009.971343,3023.545742,...,9,0,6,3,1,0,0,0,0,0


In [262]:
data=pd.concat([df,target],axis=1)

In [263]:
data.head()

,ATS0m,ATS4m,ATS3m,ATSC0m,ATS4i,ATS5i,ATS6i,ATS4v,ATS3v,ATS5v,...,nB,nC,nN,nO,nS,nP,nF,nCl,nBr,mean
identifiers,,,,,,,,,,,,,,,,,,,,,
COXSUE,2893.425299,2194.932881,2715.297217,1401.437907,5151.343678,5301.706545,6122.390295,5659.668288,6364.304724,5418.328318,...,0,7,3,1,1,0,0,0,0,7.426769
TUWFAU,2332.966431,3396.158023,3726.037218,914.989572,11123.037526,11555.150369,8815.213223,11979.313694,12065.117806,10080.901461,...,0,12,3,0,0,0,0,0,0,8.171218
DUTHEG,2400.205790,3701.031039,3905.512878,810.851632,6885.621507,7001.148205,4510.428125,8177.121069,9467.877591,5844.605240,...,0,8,5,1,0,0,0,0,0,7.449690
AXUDED08,1674.177482,1439.472399,2012.075247,644.354010,4560.942404,3048.950709,2831.649203,4327.265269,5900.133144,2875.344366,...,0,8,0,2,0,0,0,0,0,6.808171
ARAGUV,1719.285450,1388.133143,1640.528952,700.077610,5343.113326,3243.200465,3239.609099,4036.897376,5009.971343,3023.545742,...,0,6,3,1,0,0,0,0,0,7.114870


In [269]:
X_train=data.drop('mean',axis=1)
y_train=data['mean']

In [273]:
rfr=RandomForestRegressor(n_estimators=50)
rfr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [274]:
test_desc=pd.read_csv('test_descriptors.csv',index_col='identifiers')

In [275]:
X_test=test_desc[pred_column]

In [276]:
pred_rfr=rfr.predict(X_test)

In [278]:
np.savetxt('task_3_predictions.csv',pred_rfr)